# 家賃算出モデル  

トレーニングデータは以下から引用  
[「千葉商科大学 商経学部 情報コース 応用情報処理」](https://www3.cuc.ac.jp/~nagaoka/2011/ouyou/11/rent/index.html)  

- なお説明変数(X)のうち、「家の向き」は独自に数値化してみた  

初回のみインストール  

In [ ]:
!pip install numpy
!pip install pandas
!pip install japanize-matplotlib
!pip install scikit-learn


---

Python動作環境  

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.patches
import matplotlib.colors
import matplotlib.animation
from IPython.display import HTML
import japanize_matplotlib
from sklearn import linear_model


---

## トレーニングデータ：csvファイル読込  

In [ ]:
inFileName = 'rent.csv'

data = pd.read_csv(inFileName, sep = ',')
display(data.head(5))

- データの情報  

In [ ]:
data.dtypes

In [ ]:
data.info()

In [ ]:
data.describe()


---

## 各データと家賃の関係性をチェック  

- 部屋の面積 ＆ 家賃  

In [ ]:
W = np.polyfit(data['house_area'], data['monthly_price'], 1)
y_hat = np.polyval(W, data['house_area'])
W

In [ ]:
plt.figure(figsize=(7,7))
plt.scatter(data['house_area'], data['monthly_price'])
plt.plot(data['house_area'], y_hat, 'r')
plt.xlabel('面積')
plt.ylabel('家賃')
plt.grid()

- 築年数 ＆ 家賃  

In [ ]:
W = np.polyfit(data['house_age'], data['monthly_price'], 1)
y_hat = np.polyval(W, data['house_age'])
W

In [ ]:
plt.figure(figsize=(7,7))
plt.scatter(data['house_age'], data['monthly_price'])
plt.plot(data['house_age'], y_hat, 'r')
plt.xlabel('築年数')
plt.ylabel('家賃')
plt.grid()

- 駅まで歩いて何分 ＆ 家賃  

In [ ]:
W = np.polyfit(data['walk_from_station'], data['monthly_price'], 1)
y_hat = np.polyval(W, data['walk_from_station'])
W

In [ ]:
plt.figure(figsize=(7,7))
plt.scatter(data['walk_from_station'], data['monthly_price'])
plt.plot(data['walk_from_station'], y_hat, 'r')
plt.xlabel('駅徒歩分')
plt.ylabel('家賃')
plt.grid()

- 家の向き(南：3.0 ～ 北：1.0) ＆ 家賃  

In [ ]:
W = np.polyfit(data['direction_of_house'], data['monthly_price'], 1)
y_hat = np.polyval(W, data['direction_of_house'])
W

In [ ]:
plt.figure(figsize=(7,7))
plt.scatter(data['direction_of_house'], data['monthly_price'])
plt.plot(data['direction_of_house'], y_hat, 'r')
plt.xlabel('家の向き')
plt.ylabel('家賃')
plt.grid()


---

## 家賃算出モデルを作成  

- 目的変数(y)：算出結果 → 家賃  
- 説明変数(X)：算出に影響を与える項目 → 面積・築年数・駅徒歩分・家の向き  


In [ ]:
y = data['monthly_price']
X = data[['house_area', 'house_age', 'walk_from_station', 'direction_of_house']]

reg = linear_model.LinearRegression()
X = X.values
reg.fit(X, y)

家賃を算出する式  
`y = b0 + (b1 * x1) + (b2 * x2) + (b3 * x3) + (b4 * x4)` の bを表示  
以下の順番で出力される  
- b1：面積の係数  
- b2：築年数の係数  
- b3：駅徒歩分の係数  
- b4：家の向きの係数  

In [ ]:
print(reg.coef_)

- b0：切片を算出  

In [ ]:
reg.intercept_

決定係数：1に近いほど、モデルの精度が高い  

In [ ]:
reg.score(X, y)


---

## 予測：家賃を算出してみる  

- x1：面積 (小数点1桁まで入力)  
- x2：築年数 (0～)  
- x3：駅まで歩いて何分か (0～)  
- x4：家の向き (南：3.0、南東：2.7、南西：2.5、東：2.3、西：2.0、北東：1.5、北：1.0)  

In [ ]:
x1 = 17.0
x2 = 15
x3 = 20
x4 = 1.0

price = reg.predict([[x1, x2, x3, x4]])
print('¥' +'{:,.0f}'.format(int(price)))

- 北→南向きの部屋に変更  

In [ ]:
x1 = 17.0
x2 = 15
x3 = 20
x4 = 3.0    #変更

price = reg.predict([[x1, x2, x3, x4]])
print('¥' +'{:,.0f}'.format(int(price)))

- 徒歩20分 → 10分  

In [ ]:
x1 = 17.0
x2 = 15
x3 = 10    #変更
x4 = 3.0

price = reg.predict([[x1, x2, x3, x4]])
print('¥' +'{:,.0f}'.format(int(price)))

- 広い部屋：面積を25㎡へ  

In [ ]:
x1 = 25.0    #変更
x2 = 15
x3 = 10
x4 = 3.0

price = reg.predict([[x1, x2, x3, x4]])
print('¥' +'{:,.0f}'.format(int(price)))

- 徒歩10分 → 5分  

In [ ]:
x1 = 25.0
x2 = 15
x3 = 5    #変更
x4 = 3.0

price = reg.predict([[x1, x2, x3, x4]])
print('¥' +'{:,.0f}'.format(int(price)))

- 築15年 → 新築(0年)に変更  

In [ ]:
x1 = 25.0
x2 = 0    #変更
x3 = 5
x4 = 3.0

price = reg.predict([[x1, x2, x3, x4]])
print('¥' +'{:,.0f}'.format(int(price)))

-  徒歩5分 → 徒歩0分(駅近)に変更  

In [ ]:
x1 = 25.0
x2 = 0
x3 = 0    #変更
x4 = 3.0

price = reg.predict([[x1, x2, x3, x4]])
print('¥' +'{:,.0f}'.format(int(price)))


---
